In [264]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [284]:
import pandas as pd
import numpy as np
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
import os
import warnings 
import glob
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from PIL import Image
import os
from pylab import *
from utils import * 
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [266]:
influencers = [
    'mrbeast',
    ]

Application_Credentials = '/Users/aladelca/Downloads/massive-acrobat-421018-1d8b6ce1a11a.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Application_Credentials

In [267]:
data = pd.DataFrame()
df_features = pd.DataFrame()
for influencer in influencers:
    data = pd.concat([data, preprocessing(influencer)])
    data = data.reset_index(drop=True)
    df_features = pd.concat([df_features,get_features_images(influencer)])
    df_features = df_features.reset_index(drop=True)


In [268]:
df_features.to_csv('df_features.csv', index=False)

In [269]:
df_final = preprocess_features(df_features, data)

In [270]:
images = load_and_prepare_images(data)
likes = df_final['likes'].values.astype('float32')

In [271]:
x_train, x_test, y_train, y_test = train_test_split(images, likes, test_size=0.2, random_state=123)
df_train, df_test = train_test_split(df_final, test_size=0.2, random_state=123)
x_fit, x_val, y_fit, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=123)
df_fit, df_val = train_test_split(df_train, test_size=0.2, random_state=123)


In [272]:
enc = OneHotEncoder()
df_fit_enc = pd.concat([pd.DataFrame(enc.fit_transform(df_fit[['influencer']]).toarray(), 
                          columns = enc.categories_[0], 
                          index = df_fit.index),
                          df_fit.drop(columns = ['influencer'])], axis = 1)
df_val_enc = pd.concat([pd.DataFrame(enc.transform(df_val[['influencer']]).toarray(),
                          columns = enc.categories_[0],
                          index = df_val.index),
                          df_val.drop(columns = ['influencer'])], axis = 1)

df_test_enc = pd.concat([pd.DataFrame(enc.transform(df_test[['influencer']]).toarray(),
                           columns = enc.categories_[0],
                           index = df_test.index),
                           df_test.drop(columns = ['influencer'])], axis = 1)

In [273]:
df_fit_enc = df_fit_enc.fillna(0)
df_val_enc = df_val_enc.fillna(0)
df_test_enc = df_test_enc.fillna(0) 

In [274]:
### Scaling
DROP_VARS = ['likes','fecha_utc','hashtags','comentarios','key_date','filename','caption','img_source','clean_path']
esc = MinMaxScaler()
df_fit_esc = esc.fit_transform(df_fit_enc.loc[:, ~df_fit_enc.columns.isin(DROP_VARS)])
df_val_esc = esc.transform(df_val_enc.loc[:, ~df_val_enc.columns.isin(DROP_VARS)])
df_test_esc = esc.transform(df_test_enc.loc[:, ~df_test_enc.columns.isin(DROP_VARS)])


In [289]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

K.clear_session()
image_input = Input(shape=(128, 128, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(2, 2)(x)
x = MaxPooling2D(2, 2)(x)
x = Conv2D(128, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(2, 2)(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(2, 2)(x)
x = Flatten()(x)

# Entrada para la información del influencer
influencer_input = Input(shape=(df_fit_esc.shape[1],))
y = Dense(10, activation='relu')(influencer_input)  # Ajusta el tamaño según el número de categorías

# Combinar las entradas
combined = concatenate([x, y])

# Capas totalmente conectadas después de la concatenación
z = Dense(256, activation='relu')(combined)
z = Dense(1)(z)

early_stopping = EarlyStopping(monitor='val_mae', patience=50, mode='max', restore_best_weights=True)
model = Model(inputs=[image_input, influencer_input], outputs=z)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


model.fit([x_fit, df_fit_esc], y_fit, 
          epochs=1000, 
          validation_data=([x_val, df_val_esc], y_val),
          callbacks=[early_stopping])

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 783ms/step - loss: 5951757221888.0000 - mae: 1707906.0000 - val_loss: 7054166065152.0000 - val_mae: 2048421.2500
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - loss: 5951625101312.0000 - mae: 1707867.2500 - val_loss: 7053826326528.0000 - val_mae: 2048340.6250
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step - loss: 5951359811584.0000 - mae: 1707790.5000 - val_loss: 7053210288128.0000 - val_mae: 2048194.2500
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - loss: 5950879563776.0000 - mae: 1707650.5000 - val_loss: 7052213092352.0000 - val_mae: 2047957.0000
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - loss: 5950098374656.0000 - mae: 1707423.5000 - val_loss: 7050709434368.0000 - val_mae: 2047599.1250
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - loss: 5948917153792.0000 - mae: 1707081.0000 - val_loss: 7048547270656.0000 - val_mae: 2047084.0000
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - loss: 5947215839232.00

In [292]:
predictions = model.predict([x_test, df_test_esc])
print(mean_absolute_percentage_error(y_fit, model.predict([x_fit, df_fit_esc])))
print(mean_absolute_percentage_error(y_val, model.predict([x_val, df_val_esc])))
print(mean_absolute_percentage_error(y_test, predictions))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
0.9999584
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.999966
0.9999577
